#### **Exercise 5.1**
##### What is the delay for the following types of 64-bit adders?
##### Assume that each two-input gate delay is 150 ps and that a full adder delay is 450 ps.

a) Ripple-Carry adder:

$t_{ripple} = 64(450ns) = 28.8us$

b) Carry-Lookahead adder:

$t_{CLA} = 150ns + 6(150ns) + (\frac{64}{4} -1)(150ns)2 + 4(450ns) = 7.35us$

$t_{pg}$ = time to compute $P_{i}$ and $G_{i}$

$t_{pg-block}$ = time to compute entire block - num of gates*delay of gate

$(\frac{N}{k}-1)t_{and-or}$ = number of blocks the carry should pass (before the last block) * AND/OR gate delay

$kt_{fa}$ = number of bits in a block * delay of full adder


#### **Exercise 5.3**
##### Explain why a designer might choose to use a ripple-carry adder instead of a carry-lookahead adder.

Ripple carry adder uses less two-input gates, which mean using it uses less space and power consumption.

#### **Exercise 5.8
##### Design the following comparators for 32-bit unsigned numbers. Sketch the schematics.

a) not equal:

![alt text](5_8_a.png "Title")


b) greater than or equal to:

![alt text](5_8_b.png "Title")


c) less than:

![alt text](5_8_c.png "Title")
